In [1]:
#importing libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Now lets download training and test data from MNIST dataset


In [ ]:
train_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor,
    )

test_data = datasets.MNIST(
    root = "data",
    download = True,
    train = False,
    transform = ToTensor(),
    )

In [6]:
#  creating dataloaders 
train_data_loader = DataLoader(train_data,batch_size=32)
test_data_loader = DataLoader(test_data,batch_size=32)# batch size of test will remain same for all experiments
for X,y in test_data_loader :
    print(f"The image is of shape: {X.shape}")
    print(f"The shape of the label: {y.shape}")
    break

The image is of shape: torch.Size([32, 1, 28, 28])
The shape of the label: torch.Size([32])


In [ ]:
# create a base model
class SimpleNeuralNetwork(nn.module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self